In [1]:
import pandas as pd 
import os 
import numpy as np 

In [2]:
centro = '8811417' #input("Introduza o centro: ")
data_inicio ='20240101'

In [3]:

# folder_path = os.path.join(os.path.expanduser("~"), "Downloads/Pit_stop/dados_centros")

# files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]


# combined_df = pd.DataFrame()


# for file in files:
#     file_path = os.path.join(folder_path, file)
    
    
#     df = pd.read_csv(file_path, encoding="latin-1", delimiter=';')
    
    
#     combined_df = pd.concat([combined_df, df], ignore_index=True)



In [4]:
import psycopg2
import psycopg2.extras
import os
#import pygwalker as pyg

def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2024',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()
    
    
def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

In [5]:
query_filtro_data = f"SELECT *, CAST(SUBSTRING(data_criacao, 7, 4) || SUBSTRING(data_criacao, 4, 2) || SUBSTRING(data_criacao, 1, 2) AS INTEGER) AS sortable_date_int FROM expresso_2024 WHERE centro = '{centro}' AND CAST(SUBSTRING(data_criacao, 7, 4) || SUBSTRING(data_criacao, 4, 2) || SUBSTRING(data_criacao, 1, 2) AS INTEGER) > {int(data_inicio)};"
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
query = query_filtro_data
cur.execute(query)
results_expresso_ = cur.fetchall()


In [6]:
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
#query = "select CP7,longitude_x,latitude_y  from cp7coords"
query_2 = 'SELECT "CP7", "POINT_X", "POINT_Y" FROM cp7coords'
cur.execute(query_2)
df_sig = cur.fetchall()

In [7]:
# -- limpeza do dataset 
#df = combined_df
df = pd.DataFrame(results_expresso_, columns=['data_criacao', 'centro', 'Giro', 'LOPTICA', 'JANELA_HORARIA', 'NOME', 'MORADA', 'CP', 'LOCALIDADE', 'COD_T_EVEN', 'DATA_EVENTO', 'LATITUDE', 'LONGITUDE','sortable_date_int'])
df.columns=[x.lower().strip() for x in df.columns]
df.drop_duplicates()
df['centro'] = df['centro'].astype(str).str.replace('.0',"")
df['centro'] = df['centro'].str.strip()
df['giro'] = df['giro'].str.replace(' ', '').str.strip().str[:5]
df['cp'] = df['cp'].str.replace(' ', '').str.strip()
df['longitude'] = df['longitude'].str.replace('\n', '').str.strip()
df=df[df.latitude != '0']
df=df[df.latitude != '0.0']
df = df[df['latitude'] != '']
df=df[df.cod_t_even == 'I']
df = df[df['cp'].str.len() >= 8]

In [8]:
df=df[df['centro']==centro]

In [9]:

giro_unique_ocorrencias = df.groupby('giro')['data_criacao'].nunique().reset_index(name='N_dias')

loptica_unique_ocorrencias = df.groupby('giro')['loptica'].nunique().reset_index(name='N_objetos')

giros_pit_stop = pd.merge(giro_unique_ocorrencias, loptica_unique_ocorrencias, left_on='giro', right_on='giro', how='inner')

In [10]:
import pandas as pd

df['cp4_prestador'] = df['giro'].str[:4]
df['cp4_real'] = df['cp'].str[:4]

contagem_linhas = df.groupby(['cp4_prestador', 'cp4_real']).size().reset_index(name='n_objetos')

soma_total_cp4_prestador = contagem_linhas.groupby('cp4_prestador')['n_objetos'].sum()

contagem_linhas = pd.merge(contagem_linhas, soma_total_cp4_prestador, on='cp4_prestador', suffixes=('', '_total'))

contagem_filtrada = contagem_linhas[contagem_linhas['n_objetos'] > contagem_linhas['n_objetos_total'] / 15]

contagem_filtrada = contagem_filtrada.drop(columns=['n_objetos_total'])

contagem_filtrada


,cp4_prestador,cp4_real,n_objetos
0,3700,3700,6525
7,3720,3720,8871
12,3730,3730,3291
15,3750,3750,5148
23,3770,3770,2985
30,3800,3800,7533
33,3800,3810,8876
42,3830,3830,5383
49,3840,3840,2852
55,3850,3850,3882


In [11]:

# downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")

# file_path = os.path.join(downloads_path, f'{centro}.xlsx')

# with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    
#     giros_pit_stop.to_excel(writer, sheet_name='giros_pit_stop', index=False)
    
#     contagem_filtrada.to_excel(writer, sheet_name='áreas de atuação', index=False)

# print(f"File saved at: {file_path}")
